# Solutions to Tutorial 6
## Your assignment...
...should you choose to accept it will be the following:
1. Parse [this](https://en.wikipedia.org/wiki/Lists_of_stars_by_constellation) webpage for the RA and Dec of stars of each constellation, convert these coordinates to Cartesian coordinates and store them by constellation and plot them using matplotlib.
2. Try to recreate the 'Moons_and_planets.csv' file(used in the first tutorial) from [this](https://en.wikipedia.org/wiki/List_of_natural_satellites) webpage. You can take inspiration from how tables are scraped in the get_map() function for Task 1. Do remember to remove commas and uncertainties in the radius measurement.

These may not be the "best" solution to the given questions, it is just one of the many solutions possible.

## Task 1 :

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_coords(ra_s, dec_s):
    h = float(ra_s[:2])
    m = float(ra_s[3:5])
    s = float(ra_s[6:-1])
    ra = h + m/60 + s/3600
    if dec_s[0] == '+':
        sign = 1
    else:
        sign = -1
    d = float(dec_s[1:3])
    m = float(dec_s[4:6])
    s = float(dec_s[7:-1])
    dec = sign*(d + m/60 + s/3600)
    return ra, dec

In [ ]:
def get_map(constellation):        #the get_map() that skips stars
    url = f'https://en.wikipedia.org/wiki/List_of_stars_in_{constellation}'
    r = requests.get(url)

    soup = BeautifulSoup(r.content, 'lxml')

    tab = soup.find_all('table', attrs={'class':'wikitable sortable'})[0]

    data = [[]]
    for i in tab.find_all('tr'):
        row = []
        for j in i.find_all('td'):
            row.append(j.get_text())
        data.append(row)

    heads = []
    for i in tab.find_all('tr')[:1]:
        for j in i.find_all('th'):
            heads.append(j.get_text().strip('\n'))

    name_ind = heads.index('Name')
    ra_ind = heads.index('RA')
    dec_ind = heads.index('Dec')

    mag_ind = heads.index('vis.mag.')

    name = []
    ra = []
    dec = []
    mag = []
    for i in data[2:-2]:
        name_string = i[name_ind]
        try:
            ra_string = i[ra_ind].replace('\xa0', '')
            dec_string = i[dec_ind].replace('\xa0', '')
            mag_string = i[mag_ind]
            if mag_string[0]=='−':
                mag_string = '-'+mag_string[1:]
        except:
            if verb:
                print(f"{name_string} has no data for coordinates")
            continue
        try:
            ra_i, dec_i = get_coords(ra_string, dec_string)
        except:
            if verb:
                print(f"{name_string} has coordinate format issues")
            continue
        try:
            mag.append(float(mag_string))
            name.append(name_string)
            ra.append(ra_i)
            dec.append(dec_i)
        except:
            if verb:
                print(f"{name_string} does not have magnitude data")
            continue

    name = np.array(name)
    ra = np.array(ra)
    dec = np.array(dec)
    mag = np.array(mag)
    return name, ra, dec, mag

In [ ]:
def get_map(constellation):           #The get_map() that does not skip stars
    url = f'https://en.wikipedia.org/wiki/List_of_stars_in_{constellation}'
    r = requests.get(url)

    soup = BeautifulSoup(r.content, 'lxml')

    tab = soup.find_all('table', attrs={'class':'wikitable sortable'})[0]

    data = [[]]
    for i in tab.find_all('tr'):
        row = []
        for j in i.find_all('td'):
            row.append(j.get_text())
        data.append(row)

    heads = []
    for i in tab.find_all('tr')[:1]:
        for j in i.find_all('th'):
            heads.append(j.get_text().strip('\n'))

    name_ind = heads.index('Name')
    ra_ind = heads.index('RA')
    dec_ind = heads.index('Dec')

    mag_ind = heads.index('vis.mag.')

    name = []
    ra = []
    dec = []
    mag = []
    for i in data[2:-2]:
        name_string = i[name_ind]
        
        if i[ra_ind] != '' and i[dec_ind] != '' and i[mag_ind] != '' and i[ra_ind] != ' ' and i[dec_ind] != ' ' and i[mag_ind] != ' ' and i[mag_ind][0] != 'n':
            ra_string = i[ra_ind].replace('\xa0', '')
            dec_string = i[dec_ind].replace('\xa0', '')
            mag_string = i[mag_ind]
            if mag_string[0]=='−':
                mag_string = '-'+mag_string[1:]
            if mag_string[0]=='~':
                mag_string = mag_string[1:]
            mag_str = mag_string[0]
            for c in mag_string[1:]:
                if ord(c) > 45 and ord(c) < 59:
                    mag_str += c
                else:
                    break
            i = mag_string.find('.')
            mag_string = mag_string[:(i+3)]
                                
            ra_i, dec_i = get_coords(ra_string, dec_string)
            mag.append(float(mag_str))
            name.append(name_string)
            ra.append(ra_i)
            dec.append(dec_i)

    name = np.array(name)
    ra = np.array(ra)
    dec = np.array(dec)
    mag = np.array(mag)
    return name, ra, dec, mag

In [ ]:
def project(ra, dec): # Stereographic Projection. 
    theta = np.deg2rad(90-dec + dec.mean())
    phi = np.deg2rad((ra-ra.mean())*15)
    x = np.sin(theta)*np.cos(phi)
    y = np.sin(theta)*np.sin(phi)
    z = np.cos(theta)
    X = x/(1-z)
    Y = y/(1-z)
    return X,Y

In [ ]:
def plot(constellation, return_data = False, orientation=0, norm=50, **kwargs):
    name, ra, dec, mag = get_map(constellation)
    x, y = project(ra, dec)
    s = 10**(-mag/2.5)
    if norm!=None:
        s = s/s.max()*norm
    if 'figure' not in kwargs:
        plt.figure(figsize=(10,10))
    plt.gca().set_facecolor('k')
    if 'color' not in kwargs:
        kwargs['color'] = 'w'
    if orientation == 0:
        plt.scatter(x, y, s=s, **kwargs)
    elif orientation == 1:
        plt.scatter(-y, x, s=s, **kwargs)
    elif orientation == 2:
        plt.scatter(-x, -y, s=s, **kwargs)
    elif orientation == 3:
        plt.scatter(y, -x, s=s, **kwargs)
    plt.xticks([])
    plt.yticks([])
    if return_data:
        return name, ra, dec, mag, s

In [ ]:
plot('Draco',orientation=1)

## Task 2 :

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_natural_satellites")
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
table=soup.find_all('table',attrs={'class':'wikitable sortable'})[0]
table

In [ ]:
    data = [[]]
    for i in table.find_all('tr'):  #'tr' tag used to access rows
        row = []
        for j in i.find_all('td'):  #'td' tag used to acccess elements
            row.append(j.get_text().replace(',',''))  #commas removed to enable conversion of string to float
        data.append(row)

    heads = []
    for i in table.find_all('tr')[:1]:
        for j in i.find_all('th'):   #'th' used to access headers
            heads.append(j.get_text().strip('\n'))

In [ ]:
heads

In [ ]:
data

In [ ]:
final_data=[[]]
for moon in data:
    if moon!=[]:
        final_data.append([moon[3],moon[1],moon[4]])

final_data[0]=["Name of moon","Name of planet","Mean radius(km)"]   
final_data

Now, to remove the uncertainty in the radius measurements:

In [ ]:
for moon in final_data[1:-1]:
    moon[2]=moon[2].split('±')[0]    

In [ ]:
final_data